# Boolean Search in Documents
## Objective

Expand the simple term search functionality to include Boolean search capabilities. This will allow users to perform more complex queries by combining multiple search terms using Boolean operators.

## Problem Description

You must enhance the existing search engine from the previous exercise to support Boolean operators: AND, OR, and NOT. This will enable the retrieval of documents based on the logical relationships between multiple terms.

In [1]:
import os
import re
import inflect

## Requirements
### Step 1: Update Data Preparation

Ensure that the documents are still loaded and preprocessed from the previous task. The data should be clean and ready for advanced querying.

### Step 2: Create an Inverted Index

Create an inverted index from the documents. This index maps each word to the set of document IDs in which that word appears. This facilitates word lookup in the search process.

In [2]:
def crear_matriz(carpeta):
    fila_palabras = []
    matriz = []
    columna_documentos = []

    inflect_engine = inflect.engine()
    
    for archivo in os.listdir(carpeta):
        if archivo.endswith(".txt"):
            columna_documentos.append(archivo)
            print(archivo + "Primera Revision")
            ruta_archivo = os.path.join(carpeta, archivo)
            
            with open(ruta_archivo, "r", encoding="utf-8") as file:
                contenido = file.read().lower()
                
                # Utilizar expresión regular para extraer palabras
                palabras = re.findall(r'\b[a-zA-Z]+\b', contenido)
                
                # Singularizar y añadir a fila_palabras_temporal
                fila_palabras_temporal = set(inflect_engine.singular_noun(palabra) or palabra for palabra in palabras)
            
                # Singularizar y añadir a fila_palabras
                for palabra in palabras:
                    palabra_singular = inflect_engine.singular_noun(palabra) or palabra
                    if palabra_singular not in fila_palabras:
                        fila_palabras.append(palabra_singular)
    
    # Inicializar la matriz con ceros
    matriz = [[0 for _ in range(len(columna_documentos))] for _ in range(len(fila_palabras))]

    # Rellenar la matriz con la información de los documentos
    for col_idx, archivo in enumerate(columna_documentos):
        print(archivo + "Segunda Revision")
        ruta_archivo = os.path.join(carpeta, archivo)
        
        with open(ruta_archivo, "r", encoding="utf-8") as file:
            contenido = file.read().lower()
            
            # Utilizar expresión regular para extraer palabras
            palabras = re.findall(r'\b[a-zA-Z]+\b', contenido)
            
            # Singularizar palabras
            palabras_singularizadas = set(inflect_engine.singular_noun(palabra) or palabra for palabra in palabras)
            
            # Actualizar la matriz para las palabras presentes en el documento actual
            for palabra in palabras_singularizadas:
                if palabra in fila_palabras:
                    fila_idx = fila_palabras.index(palabra)
                    matriz[fila_idx][col_idx] = 1
    
    return columna_documentos, fila_palabras, matriz


### Step 3: Implementing Boolean Search

- Enhance Input Query: Modify the function to accept complex queries that can include the Boolean operators AND, OR, and NOT.
- Implement Boolean Logic:
 - AND: The document must contain all the terms. For example, python AND programming should return documents containing both "python" and "programming".
 - OR: The document can contain any of the terms. For example, python OR programming should return documents containing either "python", "programming", or both.
 - NOT: The document must not contain the term following NOT. For example, python NOT snake should return documents that contain "python" but not "snake".

In [35]:
def buscar_palabra(consulta, columna_documentos, fila_palabras, matriz):
    inflect_engine = inflect.engine()
    
    def obtener_fila_palabra(palabra):
        palabra_singular = inflect_engine.singular_noun(palabra.lower()) or palabra.lower()
        if palabra_singular in fila_palabras:
            fila_idx = fila_palabras.index(palabra_singular)
            return matriz[fila_idx]
        else:
            return [0] * len(columna_documentos)
    
    def parsear_consulta(consulta):
        consulta = consulta.upper()
        tokens = re.split(r'\s+(AND|OR|NOT)\s+', consulta)
        print(tokens)
        return tokens

    def aplicar_operacion_logica(operacion, filas):
        if operacion == "AND":
            return list(map(int, map(all, zip(*filas))))
        elif operacion == "OR":
            return list(map(int, map(any, zip(*filas))))
        elif operacion == "NOT":
            return [1 - x for x in filas[0]]
        else:
            raise ValueError(f"Operación lógica desconocida: {operacion}")
    
    tokens = parsear_consulta(consulta)
    
    # Almacena filas según las palabras
    filas = []
    operaciones = []

    for token in tokens:
        if token in {"AND", "OR", "NOT"}:
            print(token)
            operaciones.append(token)
        else:
            filas.append(obtener_fila_palabra(token))
    
    # Aplica las operaciones lógicas
    while operaciones:
        operacion = operaciones.pop(0)
        if operacion == "NOT":
            fila = filas.pop(0)
            resultado = aplicar_operacion_logica(operacion, [fila])
            filas.insert(0, resultado)
        else:
            fila1 = filas.pop(0)
            fila2 = filas.pop(0)
            resultado = aplicar_operacion_logica(operacion, [fila1, fila2])
            filas.insert(0, resultado)
    
    # Convertir el resultado a nombres de documentos
    resultado_final = filas[0]
    documentos_presentes = [columna_documentos[col_idx] for col_idx, presente in enumerate(resultado_final) if presente == 1]
    
    return documentos_presentes


In [4]:
# Carpeta donde se encuentran los archivos txt
carpeta_data = "data_muestra"

In [5]:
# Realizar diccionario con la matriz binaria
columnas_doc, fila_palabras, matriz_binaria = crear_matriz(carpeta_data)

pg11.txtPrimera Revision
pg16.txtPrimera Revision
pg23.txtPrimera Revision
pg11.txtSegunda Revision
pg16.txtSegunda Revision
pg23.txtSegunda Revision


Textos Ejemplos: pg11.txt (Mouse, Rabbit, Alice) --- pg16.txt(Mouse, Rabbit, Barrie) --- pg23.txt(Douglass)

### Step 4: Query Processing

- Parse the Query: Implement a function to parse the input query to identify the terms and operators.
- Search Documents: Based on the parsed query, implement the logic to retrieve and rank the documents according to the Boolean expressions.
- Handling Case Sensitivity and Partial Matches: Optionally, you can handle cases and partial matches to refine the search results.

In [90]:
# Pedir al usuario que ingrese la palabra a buscar
palabra_a_buscar = input("Ingrese la palabra que desea buscar: ")

Ingrese la palabra que desea buscar: Mouse AND Rabbit


In [91]:
#Realizar busqueda
archivos_encontrados = buscar_palabra(palabra_a_buscar, columnas_doc, fila_palabras, matriz_binaria)

['MOUSE', 'AND', 'RABBIT']
AND


### Step 5: Displaying Results

- Output the Results: Display the documents that match the query criteria. Include functionalities to handle queries that result in no matching documents.

In [92]:
# Se comprueba que la lista archivos_con_palabra tenga algo
if archivos_encontrados: 
    print("")
    print("Esa palabra aparece en los siguientes archivos:")
    for archivo in archivos_encontrados:
        print(archivo)
else:
    print("La palabra no se encontró en ningún archivo.")


Esa palabra aparece en los siguientes archivos:
pg11.txt
pg16.txt


### Evaluation Criteria

- Correctness: The Boolean search implementation should correctly interpret and process the queries according to the Boolean logic.
- Efficiency: Consider the efficiency of your search process, especially as the complexity of queries increases.
- User Experience: Ensure that the interface for inputting queries and viewing results is user-friendly.

### Additional Challenges (Optional)

- Nested Boolean Queries: Allow for nested queries using parentheses, such as (python OR java) AND programming.
- Phrase Searching: Implement the ability to search for exact phrases enclosed in quotes.
- Proximity Searching: Extend the search to find terms that are within a specific distance from one another.

This exercise will deepen your understanding of how search engines process and respond to complex user queries. By incorporating Boolean search, you not only enhance the functionality of your search engine but also mimic more closely how real-world information retrieval systems operate.